In [11]:
from langgraph.graph import StateGraph, START, END
from typing import List, Annotated
from dataclasses import dataclass
from langgraph.graph.message import add_messages
from langchain_core.messages import AIMessage, AnyMessage, HumanMessage

from langchain_groq import ChatGroq
from langchain_community.tools import WikipediaQueryRun, ArxivQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper
from langchain_tavily import TavilySearch

from langgraph.prebuilt import ToolNode, tools_condition
from IPython.display import display, Image
import os
from dotenv import load_dotenv

In [12]:
load_dotenv()
os.environ['TAVILY_API_KEY'] = os.getenv('TAVILY_API_KEY')

In [13]:
@dataclass
class MyastreamBot:
    messages: Annotated[List[AnyMessage], add_messages]

In [14]:
wiki_tool = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(top_k_results=5, doc_content_chars_max= 2000))
arxiv_tool = ArxivQueryRun(api_wrapper=ArxivAPIWrapper(top_k_results=5, doc_content_chars_max=2000))
tavily_tool = TavilySearch(
    max_results=5,
    topic="general"
)

tool_list = [wiki_tool,arxiv_tool,tavily_tool]

In [15]:
llm = ChatGroq(model = "qwen/qwen3-32b")
llm_bind = llm.bind_tools(tool_list)

In [16]:
def llmAgent(state: MyastreamBot):

    return {"messages": llm_bind.invoke(state.messages)}

In [17]:
def callAgent():
    graph = StateGraph(MyastreamBot)

    graph.add_node("llmAgent", llmAgent)
    graph.add_node("tools", ToolNode(tool_list))

    graph.add_edge(START, "llmAgent")
    graph.add_conditional_edges("llmAgent", tools_condition)
    graph.add_edge("tools", "llmAgent")
    graph.add_edge("llmAgent", END)

    graph_builder = graph.compile()

    return graph_builder

In [18]:
agent = callAgent()